In [1]:
import itertools as it

import mbuild as mb
import numpy as np
from mbuild.compound import Compound
from mbuild.coordinate_transform import force_overlap
from mbuild.utils.validation import assert_port_exists
from mbuild import clone
from foyer import Forcefield
from copy import deepcopy
import pytest

In [2]:
__all__ = ['Pores']

In [83]:
class Pores(mb.Compound):
    """A general slit pore recipe
    
    Parameters
    ----------
    x_sheet: int
        dimensions of graphene sheet in x-direction [nm]
    y_sheet: int
        dimensions of graphene sheet in y-direction [nm]
    sheets: int
        number of graphene sheets, default=3
    pore_width: width of slit pore [nm]
    x_bulk: length of bulk region in x-direction [nm]
    solvent: optional
        compound to solvate the system with.  If not provided, system will not be solvated
    Attributes
    ----------
    
    Notes: Match graphene y-dimension with box x-dimension
    """
    def __init__(self,x_sheet, y_sheet, sheets, pore_width, x_bulk, solvent):
        super(Pores,self).__init__()
        self.x_sheet = x_sheet
        self.y_sheet = y_sheet
        self.sheets = sheets
        self.pore_width = pore_width
        self.x_bulk = x_bulk
        
        # Do some math to figure out how much to replicate graphene cell. TODO: Figure out if rounding is necessary
        replicate = [(self.x_sheet/0.246), (self.y_sheet/0.246)*(15/13)] # multiply by (15/13) to account for later periodicity change
        if all(x <= 0 for x in [x_sheet, y_sheet]):
            raise ValueError('Dimension of graphene sheet must be greater than zero')
        self.name = 'C'
        carbon_locations = [[0,0,0], [2/3,1/3,0]]
        basis = {self.name: carbon_locations}
        graphene_lattice = mb.Lattice(lattice_spacing=[.2456,.2456,.335], angles=[90,90,120], lattice_points=basis)
        carbon = mb.Compound(name=self.name)
        graphene = graphene_lattice.populate(compound_dict={self.name: carbon},
                                         x=replicate[0],y=replicate[1],z=self.sheets)
        print(graphene.periodicity)
        for particle in graphene.particles():
            if particle.xyz[0][0] < 0:
                particle.xyz[0][0] += graphene.periodicity[0]
        self.graphene_dims = graphene.periodicity
        self.graphene_dims[1] *= 13/15
        print(self.graphene_dims)
        bottom_sheet = mb.clone(graphene)
        #changed graphene.periodicity to self.graphene_dims
        bottom_sheet.translate([0, self.pore_width+self.graphene_dims[2]-.335, 0])
        bottom_sheet.spin(1.5708,[1,0,0])
        print(bottom_sheet.n_particles)
        print(bottom_sheet.periodicity)
        self.bot_xyz = bottom_sheet.xyz
        top_sheet = mb.clone(graphene)
        top_sheet.spin(1.5708,[1,0,0])
        self.top_xyz = top_sheet.xyz
        system = mb.Compound()
        system.from_parmed(structure=bottom_sheet.to_parmed() + top_sheet.to_parmed())
        if solvent:
            self._solvate(solvent=solvent, n_compounds=1000, system=system)
            #5083 molecules
        else:
            self.add(system)
    def _solvate(self, system, solvent, n_compounds):
        """Solvate slit pore box
        Parameters
        ----------
        solvent: dictionary
            Compound and residue name to load into system {'name': compound}
        n_compounds: int
            Number of compounds to solvate with
        """
        for key, value in solvent.items():
            fluid = mb.load(value)
            fluid.name = key
        box = [(self.x_bulk*2)+self.graphene_dims[0],
                            self.pore_width+(2*(self.graphene_dims[2]-.335)),
                            self.graphene_dims[1]]

        system = mb.solvate(system, fluid, n_compounds, box=box, overlap=0.2)
        system.periodicity = box
        self.add(system)

In [84]:
water = '/Users/raymatsumoto/science/il_solvent_local/file_gen/mol2/tip3p.mol2'
test=Pores(x_sheet=3, y_sheet=3, sheets=3, pore_width=1, x_bulk=3,solvent={'tip3p': water})
test.save('init.gro',overwrite=True)

/Users/raymatsumoto/mbuild/mbuild/lattice.py:527: UserWarning: Periodicity of non-rectangular lattices are not valid with default boxes. Only rectangular lattices are valid at this time.
  warn('Periodicity of non-rectangular lattices are not valid with '


[ 2.9472  3.4384  1.005 ]
[ 2.9472      2.97994667  1.005     ]
1008
[ 2.9472      2.97994667  1.005     ]


/Users/raymatsumoto/anaconda2/envs/hackathon/lib/python3.5/site-packages/pandas/io/parsers.py:1961: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/Users/raymatsumoto/anaconda2/envs/hackathon/lib/python3.5/site-packages/pandas/io/parsers.py:1963: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [61]:
box = test.periodicity
atom_pos = test.xyz

In [65]:
#for position in atom_pos:
    #assert [element < [x for x in box] for element in position]
    #assert [element > [x for x in box] for element in position]
for position in atom_pos:
    for x in range(3):
        print(position[x])
        assert position[x] < box[x]
        
    

-0.00300000002608
3.06139993668
2.33999991417
0.0719999969006
3.06139993668
2.00500011444
0.0719999969006
3.06139993668
1.66999995708
0.0719999969006
5.8857998848
2.33999991417
0.284700006247
5.8857998848
2.00500011444
0.284700006247
5.8857998848
1.66999995708
0.284700006247
5.76300001144
2.33999991417
0.49739998579
5.76300001144
2.00500011444
0.49739998579
5.76300001144
1.66999995708
0.49739998579
5.64020013809
2.33999991417
0.710099995136
5.64020013809
2.00500011444
0.710099995136
5.64020013809
1.66999995708
0.710099995136
5.5173997879
2.33999991417
0.922800004482
5.5173997879
2.00500011444
0.922800004482
5.5173997879
1.66999995708
0.922800004482
5.39459991455
2.33999991417
1.13549995422
5.39459991455
2.00500011444
1.13549995422
5.39459991455
1.66999995708
1.13549995422
5.2718000412
2.33999991417
1.34819996357
5.2718000412
2.00500011444
1.34819996357
5.2718000412
1.66999995708
1.34819996357
5.14900016785
2.33999991417
1.56089997292
5.14900016785
2.00500011444
1.56089997292
5.14900016

In [54]:
print([element < [x for x in box] for element in position])

[array([ True, False, False], dtype=bool), array([ True,  True,  True], dtype=bool), array([ True,  True,  True], dtype=bool)]


In [80]:
x = np.max(test.bot_xyz[2]) - np.min(test.bot_xyz[2])

In [81]:
print(x)
print(np.max(test.bot_xyz[1]))

3.83595096128
3.08797746963


In [82]:
(test.periodicity[0]-x)/2

2.5556245193605989